In [2]:
import numpy as np

In [127]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegressionCV, LogisticRegression

In [171]:
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation
from keras.utils import np_utils
from keras.optimizers import SGD

# Generate Fake Data

In [342]:
n_peaks = 1000
rt_samples  = 4
rt = range(0,rt_samples * 1,1)
mz = range(0,rt_samples * 10,10)
prob = 0.5
slopes = [1000,10,10]
slope_noise = 5
rt_noise = 0.1
mz_noise = 1
intensity_noise = 300

In [343]:
peak_rt = []
peak_intensity = []
peak_outcome = []
peak_label = []
peak_mz = []
for i in range(n_peaks):
    peak_outcome.extend(np.random.binomial(1,prob,1).tolist())
    if peak_outcome[i]==1:
        peak_label.append('peak')
    else:
        peak_label.append('noise')
    peak_rt.append([rt[0]] + np.random.normal(list(rt[1:rt_samples]),0.1,rt_samples-1).tolist())
    peak_mz.append([mz[0]] + np.random.normal(list(mz[1:rt_samples]),0.1,rt_samples-1).tolist())
    if peak_outcome[i] == 1:
        b =  np.random.normal(slopes,slope_noise,3)
        peak_intensity.append(np.random.normal([b[0] + x * b[1] + x *b[2] * b[2] for x in rt],intensity_noise,rt_samples).tolist())
    else:
        b =  np.random.normal(slopes,slope_noise,3)
        peak_intensity.append(np.random.normal([b[0] for x in rt],intensity_noise,rt_samples).tolist())

In [344]:
X = np.array(peak_intensity)
y = peak_label

In [345]:
train_X, test_X, train_y, test_y = train_test_split(X, y, train_size=0.5, test_size=0.5, random_state=0)

In [346]:
def one_hot_encode_object_array(arr):
    '''One hot encode a numpy array of objects (e.g. strings)'''
    uniques, ids = np.unique(arr, return_inverse=True)
    return np_utils.to_categorical(ids, len(uniques))

train_y_ohe = one_hot_encode_object_array(train_y)
test_y_ohe = one_hot_encode_object_array(test_y)

# Logistic Regression - Fake Data

In [347]:
lr = LogisticRegression()
lr.fit(train_X, train_y)

c:\users\vinny\.virtualenvs\peakdetector-zjz7q55f\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='warn',
          n_jobs=None, penalty='l2', random_state=None, solver='warn',
          tol=0.0001, verbose=0, warm_start=False)

In [348]:
print("Accuracy = {:.2f}".format(lr.score(test_X, test_y)))

Accuracy = 0.69


In [349]:
lr_cv = LogisticRegressionCV()
lr_cv.fit(train_X, train_y)

c:\users\vinny\.virtualenvs\peakdetector-zjz7q55f\lib\site-packages\sklearn\model_selection\_split.py:2053: FutureWarning: You should specify a value for 'cv' instead of relying on the default value. The default value will change from 3 to 5 in version 0.22.
  warnings.warn(CV_WARNING, FutureWarning)


LogisticRegressionCV(Cs=10, class_weight=None, cv='warn', dual=False,
           fit_intercept=True, intercept_scaling=1.0, max_iter=100,
           multi_class='warn', n_jobs=None, penalty='l2',
           random_state=None, refit=True, scoring=None, solver='lbfgs',
           tol=0.0001, verbose=0)

In [350]:
print("Accuracy = {:.2f}".format(lr_cv.score(test_X, test_y)))

Accuracy = 0.70


# Neural Network - Fake Data

In [351]:
model = Sequential()
model.add(Dense(8, input_shape=(len(train_X[0]),)))
model.add(Activation('sigmoid'))
model.add(Dense(2))
model.add(Activation('softmax'))
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=["accuracy"])
model.fit(train_X, train_y_ohe, epochs=100, batch_size=1, verbose=0);

In [352]:
loss, accuracy = model.evaluate(test_X, test_y_ohe, verbose=0)
print("Accuracy = {:.2f}".format(accuracy))

Accuracy = 0.60
